In [2]:
import pandas as pd
import streamlit as st
import numpy as np

In [3]:
league = 'SBC'
season = 2038
curr_date = '2038-11-01'
league_al_name = 'AL' 
league_nl_name = 'NL'

# NBC Tournament only. This needs to be updated once annually
tourney_start = '2023-07-04'
tourney_end = '2023-07-25'

path = 'C:/Users/night/OneDrive/Documents/Out of the Park Developments/OOTP Baseball 21/saved_games/'

In [4]:
leagues = {
    "SBC":[145, 
           '2038-04-04', 
           2038,
           path + 'SBC.lg/import_export/csv/'],
    "PBC":[146, 
           '2030-10-28',
           2030,
           path + 'PBC.lg/import_export/csv/'],
    "NBC":[153, 
           '2023-04-11',
           2024,
           path + 'NBC.lg/import_export/csv/'],
    "MVP":[153, 
           '2024-03-28', 
           2024,
           path + 'MVP.lg/import_export/csv/']
}

In [9]:
def comp_balance(league):
    team_history = pd.read_csv(leagues[league][3] + 'team_history.csv')
    team_history_financials = pd.read_csv(leagues[league][3] + 'team_history_financials.csv')
    team_history = team_history[(team_history['year'] == (leagues[league][2] - 1)) & 
                            (team_history['league_id'] == leagues[league][0])].set_index('team_id')

    team_history = team_history[['name', 'nickname', 'made_playoffs']]

    team_history_financials = team_history_financials[(team_history_financials['year'] == (leagues['SBC'][2] - 1)) & 
                            (team_history_financials['league_id'] == leagues['SBC'][0])].set_index('team_id')

    temp = pd.merge(team_history, team_history_financials, how = 'left', left_index = True, right_index = True)
    
    comp_bal = pd.DataFrame()
    comp_bal['Team'] = temp['name'] + ' ' + temp['nickname']
    comp_bal['Total Revenue'] = temp['gate_revenue'] + \
                                temp['season_ticket_revenue'] + \
                                temp['media_revenue'] + \
                                temp['merchandising_revenue'] + \
                                temp['playoff_revenue']
    comp_bal['Media Revenue'] = temp['media_revenue']
    comp_bal['Media Revenue %'] = round((comp_bal['Media Revenue']/comp_bal['Total Revenue'])*100,1)
    comp_bal['Other Revenue %'] = 100 - comp_bal['Media Revenue %']
    comp_bal['Market Size'] = temp['market']
    comp_bal['Playoffs'] = temp['made_playoffs']
    
    return comp_bal

sbc_comp_bal = comp_balance('SBC')
st.table(sbc_comp_bal)

In [6]:
def teams(league):
    teams = pd.read_csv(leagues[league][3] + 'teams.csv')[['name', 'nickname','team_id','sub_league_id', 'division_id']]
    teams = teams[teams['team_id'] < 80]
    teams['league'] = np.where(teams['sub_league_id'] == 0, "AL", "NL")
    teams['division'] = np.select(
    [
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 0),
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 1),
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 2),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 0),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 1),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 2)
    ],
    [
        'East',
        'Central',
        'West',
        'East',
        'Central',
        'West'
    ],
    default = 'Unknown'
    )

    teams['team'] = teams['name'] + ' ' + teams['nickname']

    return teams[['team', 'league', 'division']].set_index(teams['team_id'])

In [10]:
home_games = pd.read_csv(leagues[league][3] + 'games.csv')
home_games = home_games[(home_games['league_id'] == leagues[league][0]) & 
                        (home_games['game_type'] == 0) &
                        (home_games['date'].between(leagues[league][1], curr_date, inclusive = True))]
home_games['home_id'] = home_games['home_team']
home_games['runs_home'] = home_games['runs1']
home_games['hits_home'] = home_games['hits1']
home_games['errors_home'] = home_games['errors1']

home_games = home_games[['home_id','date', 'runs_home', 'hits_home', 'errors_home', 'innings']].set_index(home_games['game_id'])


away_games = pd.read_csv(leagues[league][3] + 'games.csv')
away_games = away_games[(away_games['league_id'] == leagues[league][0]) & 
                        (away_games['game_type'] == 0) &
                        (away_games['date'].between(leagues[league][1], curr_date, inclusive = True))]
away_games['away_id'] = away_games['away_team']
away_games['runs_away'] = away_games['runs0']
away_games['hits_away'] = away_games['hits0']
away_games['errors_away'] = away_games['errors0']

away_games = away_games[['away_id','date', 'runs_away', 'hits_away', 'errors_away', 'innings']].set_index(away_games['game_id'])

games_columns = away_games.columns.difference(home_games.columns)

games = home_games.merge(away_games[games_columns], on = 'game_id').merge(
    teams(league), how = 'left', left_on = 'home_id', right_on = 'team_id').rename(
    columns={'team':'Home_Team', 'league':'Home_League', 'division':'Home_Division'}).merge(
    teams(league), how = 'left', left_on = 'away_id', right_on = 'team_id').rename(
    columns={'team':'Away_Team', 'league':'Away_League', 'division':'Away_Division'})

standings = games.copy()
standings['home_win'] = np.where(standings['runs_home'] > standings['runs_away'],1,0)
standings['home_loss'] = np.where(standings['runs_home'] < standings['runs_away'],1,0)
standings['away_win'] = np.where(standings['runs_home'] < standings['runs_away'],1,0)
standings['away_loss'] = np.where(standings['runs_home'] > standings['runs_away'],1,0)

standings['home_div_win'] = np.where(
    (standings['Home_Division'] == standings['Away_Division']) &
    (standings['Home_League'] == standings['Away_League']) &
    (standings['runs_home'] > standings['runs_away']),1,0)

standings['home_div_loss'] = np.where(
    (standings['Home_Division'] == standings['Away_Division']) &
    (standings['Home_League'] == standings['Away_League']) &
    (standings['runs_home'] < standings['runs_away']),1,0)

standings['away_div_win'] = np.where(
    (standings['Home_Division'] == standings['Away_Division']) &
    (standings['Home_League'] == standings['Away_League']) &
    (standings['runs_home'] < standings['runs_away']),1,0)

standings['away_div_loss'] = np.where(
    (standings['Home_Division'] == standings['Away_Division']) &
    (standings['Home_League'] == standings['Away_League']) &
    (standings['runs_home'] > standings['runs_away']),1,0)

In [8]:
standings

,home_id,date,runs_home,hits_home,errors_home,innings,away_id,errors_away,hits_away,runs_away,...,Away_League,Away_Division,home_win,home_loss,away_win,away_loss,home_div_win,home_div_loss,away_div_win,away_div_loss
0,36,2038-10-1,5,11,1,9,38,0,14,12,...,AL,East,0,1,1,0,0,0,0,0
1,49,2038-10-1,4,12,0,11,41,0,8,2,...,NL,Central,1,0,0,1,1,0,0,1
2,33,2038-10-1,4,11,1,9,53,0,8,3,...,AL,East,1,0,0,1,1,0,0,1
3,32,2038-10-1,10,14,0,9,60,1,4,3,...,NL,Central,1,0,0,1,1,0,0,1
4,34,2038-10-1,3,10,1,9,57,0,10,7,...,AL,East,0,1,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,48,2038-10-6,2,7,1,9,33,1,2,0,...,AL,East,1,0,0,1,1,0,0,1
80,50,2038-10-6,7,15,1,9,49,0,6,3,...,NL,Central,1,0,0,1,1,0,0,1
81,38,2038-10-6,5,14,0,14,47,0,10,6,...,AL,East,0,1,1,0,0,1,1,0
82,37,2038-10-6,5,7,0,9,52,0,10,4,...,NL,Central,1,0,0,1,1,0,0,1
